# Creamos un agente muy simple (con Tracing)

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

True

## Primero creamos una conexión con OpenAI

In [2]:
# Necesitamos importar la API Key de OpenAI
if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set.  """
    )


In [5]:
# Comprobamos acceso a OpenAI
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], 
        input="Say: Conectado!"
    )
    .output_text
)

Conectado!


# Luego importamos librerias para crear el Agente

In [7]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

### Runner Class
The Runner class from the agents library is the executor/orchestrator that runs your AI agent and manages the interaction with the language model. 

It has two main methods:

**1. Runner.run() - Synchronous/Complete Response**

Executes the agent and waits for the complete response:

```Python
result = await Runner.run(nutrition_agent, "How healthy are bananas?")
```

**Returns**: A RunResult object containing:
- The complete final output
- Agent metadata
- Raw responses from the model
- Number of items processed
- Guardrail results (if any)

**2. Runner.run_streamed() - Streaming Response**

Executes the agent and streams the response token by token (word by word):

```Python
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are apples?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True
```

**Returns**: A stream object that yields events in real-time, useful for showing progressive responses to the user.

Creamos un agente sencillo de nombre "Nutrition Assistant"

In [13]:
# Creamos el agente
agente_simple = Agent(
    name="Asistente de Nutrición",
    instructions="""
    Eres un asistente que proporciona información acerca de ciertos alimentos.
    """
    )


Ejecutemos el agente con el Runner de forma asincrona:

In [15]:
# Lo ejecutamos asíncronamente. El agente responde a la pregunta sobre cuán saludables son los plátanos.  
with trace("Agente simple"):
    result = await Runner.run(agente_simple, "Los platanos son saludables?")

print(result)

RunResult:
- Last agent: Agent(name="Asistente de Nutrición", ...)
- Final output (str):
    Sí, los plátanos son saludables cuando se consumen con moderación dentro de una dieta balanceada. Beneficios principales:
    - Alto contenido de potasio, que ayuda a la función cardíaca y la presión arterial.
    - Buena fuente de vitaminas B6 y C, fibra y energía rápida.
    - Opción natural y portátil, sin grasa.
    
    Consideraciones:
    - Contienen azúcares naturales; si tienes diabetes, controla la porción.
    - Personas con problemas renales deben consultar sobre la ingesta de potasio.
    - El nivel de madurez afecta el contenido de azúcar: más maduros, más azúcares simples.
    
    En resumen: son una opción nutritiva y versátil, adecuada para la mayoría.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


-------
Podemos hacer que la respuesta sea en modo streaming... es decir token por token y no esperar a tener toda la respuesta.

In [16]:
response_stream = Runner.run_streamed(agente_simple, "Los torreznos son saludables?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

En general, los torreznos no se consideran un alimento “saludable” si se comen con frecuencia.

Razones:
- Muy altos en grasa y calorías.
- Contienen grasas saturadas y sodio (dependiendo de la receta).
- Pocas vitaminas o minerales en comparación con otros alimentos.

Consejos si decides comerlos:
- Consúmelos con moderación y dentro de una dieta equilibrada.
- Prefiere métodos de cocción que reduzcan grasa, como hornear o escaldar y luego dorar, y escurrir bien el aceite.
- Acompáñalos de verduras y proteína magra para equilibrio.
- Si tienes colesterol alto o hipertensión, conviene limitarlos.

¿Quieres ideas para una versión más ligera o para una alternativa más sana?